# Step.0 Prepare

In [10]:
# 1. Load speaker embedding
import glob
import numpy as np
from scipy import spatial
import pickle

embds_dict={}
for npy_path in glob.glob('./DRL-TU/egs/embed/time_varying_all_T_epoch21_rank*.npy'):
    print(npy_path)
    embds_dict_tmp = np.load(npy_path,allow_pickle=True).item()
    embds_dict ={**embds_dict,**embds_dict_tmp}

./DRL-TU/egs/embed/time_varying_all_T_epoch21_rank0.npy
./DRL-TU/egs/embed/time_varying_all_T_epoch21_rank1.npy
./DRL-TU/egs/embed/time_varying_all_T_epoch21_rank2.npy


# Step.1 Inference

In [11]:
from collections import defaultdict
from utils.util import compute_eer
from tqdm import tqdm
embd_size=128
scenario2model2day2score_v0=defaultdict(list)
scenario2model2day2score_v1=defaultdict(list)
scenario2model2day2score_v21=defaultdict(list)
scenario2model2day2score_v22=defaultdict(list)

## Baseline Method  Scoring ( V0 )

In [12]:
eers_v0={}
mindcts_v0={}

for tmp_scenario in tqdm(['random','1das','3das','5das','10das','1d1s','3d1s','5d1s','10d1s']):
    # 0. init
    scenario2model2day2score_v0[tmp_scenario]=defaultdict(list)
    
    # 1. Load trial trajectory and enrollment model
    f_read = open('./trials/%s/trial.pkl' %tmp_scenario, 'rb')
    trials = pickle.load(f_read)
    f_read.close()
    enrol_models={i.split()[0]:i.split()[1:] for i in open('./trials/%s/enrol_model'%tmp_scenario)}
    
    # 2. achieve enrol template
    enrol_embd_dict={}
    for tmp_model in enrol_models:
        enrol_embd=np.zeros((1,128))
        for utt in enrol_models[tmp_model]:
            enrol_embd += embds_dict[utt]

        enrol_embd_dict[tmp_model]=enrol_embd/np.linalg.norm(enrol_embd)
    # 3. scoring
    true_score=[]
    false_score=[]
    
    if tmp_scenario in ['random']:
        for tmp_model in trials:
            ts=tmp_model[3:7]
            scenario2model2day2score_v0[tmp_scenario][tmp_model]=defaultdict(list)
            for utt in trials[tmp_model]:
                test_embd = embds_dict[utt]
                result = 1 - spatial.distance.cosine(enrol_embd_dict[tmp_model], test_embd)
                if ts == utt[:4]:
                    true_score.append(result) 
                    scenario2model2day2score_v0[tmp_scenario][tmp_model]['true'].append(result)
                else:
                    false_score.append(result) 
                    scenario2model2day2score_v0[tmp_scenario][tmp_model]['false'].append(result)
    elif tmp_scenario in ['1das','3das','5das','10das']:
        for tmp_model in trials:
            ts=tmp_model[3:7]
            scenario2model2day2score_v0[tmp_scenario][tmp_model]=defaultdict(list)
            for day in trials[tmp_model]:
                scenario2model2day2score_v0[tmp_scenario][tmp_model][day]=defaultdict(list)
                for utt in trials[tmp_model][day]:
                    test_embd = embds_dict[utt]
                    result = 1 - spatial.distance.cosine(enrol_embd_dict[tmp_model], test_embd)
                    if ts == utt[:4]:
                        true_score.append(result) 
                        scenario2model2day2score_v0[tmp_scenario][tmp_model][day]['true'].append(result)
                    else:
                        false_score.append(result) 
                        scenario2model2day2score_v0[tmp_scenario][tmp_model][day]['false'].append(result)
    elif tmp_scenario in ['1d1s','3d1s','5d1s','10d1s']:
        for tmp_round in trials:
            scenario2model2day2score_v0[tmp_scenario][tmp_round]=defaultdict(list)
            for tmp_model in trials[tmp_round]:
                ts=tmp_model[3:7]
                scenario2model2day2score_v0[tmp_scenario][tmp_round][tmp_model]=defaultdict(list)
                for day in trials[tmp_round][tmp_model]:
                    scenario2model2day2score_v0[tmp_scenario][tmp_round][tmp_model][day]=defaultdict(list)
                    for utt in trials[tmp_round][tmp_model][day]:
                        test_embd = embds_dict[utt]
                        result = 1 - spatial.distance.cosine(enrol_embd_dict[tmp_model], test_embd)
                        if ts == utt[:4]:
                            true_score.append(result) 
                            scenario2model2day2score_v0[tmp_scenario][tmp_round][tmp_model][day]['true'].append(result)
                        else:
                            false_score.append(result) 
                            scenario2model2day2score_v0[tmp_scenario][tmp_round][tmp_model][day]['false'].append(result)

    eer_v0, threshold_eer, mindct_v0, threashold_dct = compute_eer(np.array(true_score), np.array(false_score))
    eers_v0[tmp_scenario]=eer_v0
    mindcts_v0[tmp_scenario]=mindct_v0
    print('---- Scenario : %s----'%tmp_scenario)
    print('EER: %s, EER_threshold: %s \nMinDCT: %s, MinDCT_threshold: %s, '%(eer_v0*100,threshold_eer, mindct_v0, threashold_dct))
    print(len(true_score),len(false_score))
    

 11%|██████████████████████████▎                                                                                                                                                                                                                  | 1/9 [00:09<01:15,  9.44s/it]

---- Scenario : random----
EER: 3.741261927387755, EER_threshold: 0.4204883810721398 
MinDCT: 0.4168760332730183, MinDCT_threshold: 0.6114183805549603, 
76819 76819


 22%|████████████████████████████████████████████████████▋                                                                                                                                                                                        | 2/9 [00:18<01:05,  9.40s/it]

---- Scenario : 1das----
EER: 3.5488372699646553, EER_threshold: 0.42262014935163994 
MinDCT: 0.4114094922593351, MinDCT_threshold: 0.6101848045173396, 
76673 76673


 33%|███████████████████████████████████████████████████████████████████████████████                                                                                                                                                              | 3/9 [00:29<01:00, 10.03s/it]

...

---- Scenario : 3d1s----
EER: 3.214285714285714, EER_threshold: 0.43185900051725 
MinDCT: 0.38895053166897825, MinDCT_threshold: 0.622124760633776, 
86520 86520


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 8/9 [01:17<00:10, 10.08s/it]

---- Scenario : 5d1s----
EER: 3.1555971312753353, EER_threshold: 0.4349993243156891 
MinDCT: 0.4003429996881821, MinDCT_threshold: 0.6009552607194296, 
64140 64140


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:25<00:00,  9.51s/it]

---- Scenario : 10d1s----
EER: 3.0926594464500603, EER_threshold: 0.43475103455404407 
MinDCT: 0.3838748495788207, MinDCT_threshold: 0.6196905461999074, 
58170 58170


## FixW-TU  Scoring ( V1 )

In [13]:
eers_v1=defaultdict(list)
mindcts_v1=defaultdict(list)
Threshold=0.51

for alpha in [0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
    scenario2model2day2score_v1[alpha]=defaultdict(list)
    eers_v1[alpha]={}
    mindcts_v1[alpha]={}
    for tmp_scenario in tqdm(['random','1das','3das','5das','10das','1d1s','3d1s','5d1s','10d1s']):
        # 0. init
        scenario2model2day2score_v1[alpha][tmp_scenario]=defaultdict(list)

        # 1. Load trial trajectory and enrollment model
        f_read = open('./trials/%s/trial.pkl' %tmp_scenario, 'rb')
        trials = pickle.load(f_read)
        f_read.close()
        enrol_models={i.split()[0]:i.split()[1:] for i in open('./trials/%s/enrol_model'%tmp_scenario)}

        # 2. achieve enrol template
        enrol_embd_dict={}
        for tmp_model in enrol_models:
            enrol_embd=np.zeros((1,128))
            for utt in enrol_models[tmp_model]:
                enrol_embd += embds_dict[utt]

#             enrol_embd_dict[tmp_model]=enrol_embd/np.linalg.norm(enrol_embd)
            enrol_embd_dict[tmp_model]=enrol_embd/len(enrol_models[tmp_model])
        # 3. scoring
        true_score=[]
        false_score=[]

        if tmp_scenario in ['random']:
            for tmp_model in trials:
                enrol_embd = enrol_embd_dict[tmp_model]
                ts=tmp_model[3:7]
                scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model]=defaultdict(list)
                for utt in trials[tmp_model]:
                    test_embd = embds_dict[utt]
                    result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                    if ts == utt[:4]:
                        true_score.append(result) 
                        scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model]['true'].append(result)
                    else:
                        false_score.append(result) 
                        scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model]['false'].append(result)
                    
                    if result> Threshold:
                        enrol_embd = (1-alpha)*enrol_embd + alpha*test_embd
                    
        elif tmp_scenario in ['1das','3das','5das','10das']:
            for tmp_model in trials:
                enrol_embd = enrol_embd_dict[tmp_model]
                ts=tmp_model[3:7]
                scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model]=defaultdict(list)
                for day in trials[tmp_model]:
                    scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model][day]=defaultdict(list)
                    for utt in trials[tmp_model][day]:
                        test_embd = embds_dict[utt]
                        result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                        if ts == utt[:4]:
                            true_score.append(result) 
                            scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model][day]['true'].append(result)
                        else:
                            false_score.append(result) 
                            scenario2model2day2score_v1[alpha][tmp_scenario][tmp_model][day]['false'].append(result)
                        
                        if result> Threshold:
                            enrol_embd = (1-alpha)*enrol_embd + alpha*test_embd
                        
        elif tmp_scenario in ['1d1s','3d1s','5d1s','10d1s']:
            for tmp_round in trials:
                scenario2model2day2score_v1[alpha][tmp_scenario][tmp_round]=defaultdict(list)
                for tmp_model in trials[tmp_round]:
                    enrol_embd = enrol_embd_dict[tmp_model]
                    ts=tmp_model[3:7]
                    scenario2model2day2score_v1[alpha][tmp_scenario][tmp_round][tmp_model]=defaultdict(list)
                    for day in trials[tmp_round][tmp_model]:
                        scenario2model2day2score_v1[alpha][tmp_scenario][tmp_round][tmp_model][day]=defaultdict(list)
                        for utt in trials[tmp_round][tmp_model][day]:
                            test_embd = embds_dict[utt]
                            result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                            if ts == utt[:4]:
                                true_score.append(result) 
                                scenario2model2day2score_v1[alpha][tmp_scenario][tmp_round][tmp_model][day]['true'].append(result)
                            else:
                                false_score.append(result) 
                                scenario2model2day2score_v1[alpha][tmp_scenario][tmp_round][tmp_model][day]['false'].append(result)
                            
                            if result> Threshold:
                                enrol_embd = (1-alpha)*enrol_embd + alpha*test_embd
                                
        eer_v1, threshold_eer, mindct_v1, threashold_dct = compute_eer(np.array(true_score), np.array(false_score))
        eers_v1[alpha][tmp_scenario]=eer_v1
        mindcts_v1[alpha][tmp_scenario]=mindct_v1

        print('---- Alpha: %s, Scenario : %s----'%(alpha,tmp_scenario))
        print('EER: %s, EER_threshold: %s \nMinDCT: %s, MinDCT_threshold: %s, '%(eer_v1*100,threshold_eer, mindct_v1, threashold_dct))
        print(len(true_score),len(false_score))
    

 11%|██████████████████████████▎                                                                                                                                                                                                                  | 1/9 [00:10<01:21, 10.17s/it]

---- Alpha: 0.05, Scenario : random----
EER: 2.360093206107864, EER_threshold: 0.472695597257236 
MinDCT: 0.3099103086476002, MinDCT_threshold: 0.6543091019062975, 
76819 76819


 22%|████████████████████████████████████████████████████▋                                                                                                                                                                                        | 2/9 [00:20<01:10, 10.09s/it]

---- Alpha: 0.05, Scenario : 1das----
EER: 1.1620779153026488, EER_threshold: 0.5219353950380302 
MinDCT: 0.23469800320843062, MinDCT_threshold: 0.6626917801232193, 
76673 76673


 33%|███████████████████████████████████████████████████████████████████████████████                                                                                                                                                              | 3/9 [00:31<01:04, 10.71s/it]

...

---- Alpha: 0.5, Scenario : 3d1s----
EER: 2.7288488210818307, EER_threshold: 0.44455355359935356 
MinDCT: 0.37357836338418865, MinDCT_threshold: 0.6267795874983382, 
86520 86520


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 8/9 [01:23<00:10, 10.34s/it]

---- Alpha: 0.5, Scenario : 5d1s----
EER: 2.8235110695353915, EER_threshold: 0.44377382846249636 
MinDCT: 0.38372310570626755, MinDCT_threshold: 0.6195593052108259, 
64140 64140


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:33<00:00, 10.39s/it]

---- Alpha: 0.5, Scenario : 10d1s----
EER: 2.905277634519512, EER_threshold: 0.44052119023018776 
MinDCT: 0.38012721334020977, MinDCT_threshold: 0.6107626067711185, 
58170 58170


## DRL-TU-AdW (V21)

In [16]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class Pi_net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Pi_net, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, int(hidden_size/2))
        self.mu = nn.Linear(int(hidden_size/2), output_size)
        self.sigma = nn.Linear(int(hidden_size/2), output_size)

    def forward(self, s):
        x = F.relu(self.linear1(s))
        x = F.relu(self.linear2(x))
        mu = (torch.tanh(self.mu(x))+1)/2
        sigma = F.softplus(self.mu(x))
        return mu,sigma 

actor = Pi_net(256, 256, 1) # 
checkpoint = torch.load('./DRL-TU/egs/exp/DRL-TU-AdW_pretrained/model_final.pkl',map_location='cpu')
actor.load_state_dict(checkpoint['pi'],strict=False)
# actor = actor.to('cuda:3')

_IncompatibleKeys(missing_keys=[], unexpected_keys=['linear_determine.weight', 'linear_determine.bias'])

In [ ]:
eers_v21={}
mindcts_v21={}
Threshold=0.51
visual=False

for tmp_scenario in tqdm(['random','1das','3das','5das','10das','1d1s','3d1s','5d1s','10d1s']):
# for tmp_scenario in tqdm(['random']):
    # 0. init
    scenario2model2day2score_v21[tmp_scenario]=defaultdict(list)
    
    # 1. Load trial trajectory and enrollment model
    f_read = open('./trials/%s/trial.pkl' %tmp_scenario, 'rb')
    trials = pickle.load(f_read)
    f_read.close()
    enrol_models={i.split()[0]:i.split()[1:] for i in open('./trials/%s/enrol_model'%tmp_scenario)}
    
    # 2. achieve enrol template
    enrol_embd_dict={}
    for tmp_model in enrol_models:
        enrol_embd=np.zeros((1,128))
        for utt in enrol_models[tmp_model]:
            enrol_embd += embds_dict[utt]

#         enrol_embd_dict[tmp_model]=enrol_embd/np.linalg.norm(enrol_embd)
        enrol_embd_dict[tmp_model]=enrol_embd/5
    # 3. scoring
    true_score=[]
    false_score=[]
    
    if tmp_scenario in ['random']:
        for tmp_model in trials:
            enrol_embd = enrol_embd_dict[tmp_model]
            ts=tmp_model[3:7]
            scenario2model2day2score_v21[tmp_scenario][tmp_model]=defaultdict(list)
            for utt in trials[tmp_model]:
                test_embd = embds_dict[utt]
                
                result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                
                if ts == utt[:4]:
                    true_score.append(result) 
                    scenario2model2day2score_v21[tmp_scenario][tmp_model]['true'].append(result)
                else:
                    false_score.append(result) 
                    scenario2model2day2score_v21[tmp_scenario][tmp_model]['false'].append(result)
                    
                if result> Threshold:
                    enrol_embd = enrol_embd/np.linalg.norm(enrol_embd)
                    test_embd = test_embd/np.linalg.norm(test_embd)
                    
                    observation = np.concatenate((enrol_embd,test_embd),axis=1)
                    observation = torch.from_numpy(observation.astype('float32'))
#                     observation = observation.to('cuda:3')
                    mu,sigma = actor(observation)
                    action_value = mu
                    action_value = action_value.cpu().detach().cpu().numpy()[0][0]
                    if visual: print(utt,ts,result,action_value)
                    enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd
                    
    elif tmp_scenario in ['1das','3das','5das','10das']:
        for tmp_model in trials:
            enrol_embd = enrol_embd_dict[tmp_model]
            ts=tmp_model[3:7]
            scenario2model2day2score_v21[tmp_scenario][tmp_model]=defaultdict(list)
            for day in trials[tmp_model]:
                scenario2model2day2score_v21[tmp_scenario][tmp_model][day]=defaultdict(list)
                for utt in trials[tmp_model][day]:
                    test_embd = embds_dict[utt]
                    
                    result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                    if ts == utt[:4]:
                        true_score.append(result) 
                        scenario2model2day2score_v21[tmp_scenario][tmp_model][day]['true'].append(result)
                    else:
                        false_score.append(result) 
                        scenario2model2day2score_v21[tmp_scenario][tmp_model][day]['false'].append(result)
                        
                    if result> Threshold:
                        enrol_embd = enrol_embd/np.linalg.norm(enrol_embd)
                        test_embd = test_embd/np.linalg.norm(test_embd)
                        
                        observation = np.concatenate((enrol_embd,test_embd),axis=1)
                        observation = torch.from_numpy(observation.astype('float32'))
#                         observation = observation.to('cuda:3')
                        mu,sigma = actor(observation)
                        action_value = mu
                        action_value = action_value.cpu().detach().cpu().numpy()[0][0]
                        if visual: print(utt,ts,result,action_value)
                        enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd
                        
    elif tmp_scenario in ['1d1s','3d1s','5d1s','10d1s']:
        for tmp_round in trials:
            scenario2model2day2score_v21[tmp_scenario][tmp_round]=defaultdict(list)
            for tmp_model in trials[tmp_round]:
                enrol_embd = enrol_embd_dict[tmp_model]
                ts=tmp_model[3:7]
                scenario2model2day2score_v21[tmp_scenario][tmp_round][tmp_model]=defaultdict(list)
                for day in trials[tmp_round][tmp_model]:
                    scenario2model2day2score_v21[tmp_scenario][tmp_round][tmp_model][day]=defaultdict(list)
                    for utt in trials[tmp_round][tmp_model][day]:
                        test_embd = embds_dict[utt]
                        
                        result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                        if ts == utt[:4]:
                            true_score.append(result) 
                            scenario2model2day2score_v21[tmp_scenario][tmp_round][tmp_model][day]['true'].append(result)
                        else:
                            false_score.append(result) 
                            scenario2model2day2score_v21[tmp_scenario][tmp_round][tmp_model][day]['false'].append(result)
                            
                        if result> Threshold:
                            enrol_embd = enrol_embd/np.linalg.norm(enrol_embd)
                            test_embd = test_embd/np.linalg.norm(test_embd)
                            
                            observation = np.concatenate((enrol_embd,test_embd),axis=1)
                            observation = torch.from_numpy(observation.astype('float32'))
#                             observation = observation.to('cuda:3')
                            mu,sigma = actor(observation)
                            action_value = mu
                            action_value = action_value.cpu().detach().cpu().numpy()[0][0]
                            if visual: print(utt,ts,result,action_value)
                            enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd

    eer_v21, threshold_eer, mindct_v21, threashold_dct = compute_eer(np.array(true_score), np.array(false_score))
    eers_v21[tmp_scenario]=eer_v21
    mindcts_v21[tmp_scenario]=mindct_v21
    print('---- Scenario : %s----'%tmp_scenario)
    print('EER: %s, EER_threshold: %s \nMinDCT: %s, MinDCT_threshold: %s, '%(eer_v21*100,threshold_eer, mindct_v21, threashold_dct))
    print(len(true_score),len(false_score))
    

 11%|██████████████████████████▎                                                                                                                                                                                                                  | 1/9 [00:59<07:56, 59.54s/it]

---- Scenario : random----
EER: 2.082818052825473, EER_threshold: 0.4823680352742584 
MinDCT: 0.29384657441518375, MinDCT_threshold: 0.6615565213392994, 
76819 76819


 22%|████████████████████████████████████████████████████▋                                                                                                                                                                                        | 2/9 [01:59<06:57, 59.58s/it]

---- Scenario : 1das----
EER: 1.139905833865898, EER_threshold: 0.5213414858432526 
MinDCT: 0.2349718936261787, MinDCT_threshold: 0.6590617168880055, 
76673 76673


 33%|███████████████████████████████████████████████████████████████████████████████                                                                                                                                                              | 3/9 [03:09<06:28, 64.73s/it]

...

---- Scenario : 1d1s----
EER: 1.6525965720133027, EER_threshold: 0.49847401919951717 
MinDCT: 0.27421335379892553, MinDCT_threshold: 0.6432642514577953, 
78180 78180


 78%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                    | 7/9 [06:44<01:54, 57.35s/it]

---- Scenario : 3d1s----
EER: 2.2850208044382803, EER_threshold: 0.4704318817183428 
MinDCT: 0.31716366158113735, MinDCT_threshold: 0.6396442823719398, 
86520 86520


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 8/9 [07:30<00:53, 53.87s/it]

---- Scenario : 5d1s----
EER: 2.513252260679763, EER_threshold: 0.4621256025759285 
MinDCT: 0.3482850015590895, MinDCT_threshold: 0.6146218773801573, 
64140 64140


## DRL-TU-MH (V22)

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def init(module, weight_init, bias_init, gain=1):
    weight_init(module.weight.data, gain=gain)
    bias_init(module.bias.data)
    return module

class AddBias(nn.Module):
    def __init__(self, bias):
        super(AddBias, self).__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))

    def forward(self, x):
        bias = self._bias.t().view(1, -1)
        return x + bias

#Categorical
class FixedCategorical(torch.distributions.Categorical):
    def sample(self):
        return super().sample().unsqueeze(-1)

    def log_probs(self, actions):
        return super().log_prob(actions.squeeze(-1)).view(actions.size(0), -1).sum(-1).unsqueeze(-1)

    def entropy(self):
        p = self.probs.masked_fill(self.probs <= 0, 1)
        return -1 * p.mul(p.log()).sum(-1)

    def mode(self):
        return self.probs.argmax(dim=-1, keepdim=True)

class Categorical(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Categorical, self).__init__()

        init_ = lambda m: init(
            m, nn.init.orthogonal_, lambda x: nn.init.constant_(x, 0), gain=0.01
        )

        self.linear = init_(nn.Linear(num_inputs, num_outputs))

    def forward(self, x, mask=None):
        x = self.linear(x)
        if mask is not None:
            return FixedCategorical(logits=x + torch.log(mask))
        else:
            return FixedCategorical(logits=x)

#Normal
class FixedNormal(torch.distributions.Normal):
    def log_probs(self, actions):
        return super().log_prob(actions).sum(-1, keepdim=True)

    def entropy(self):
        return super().entropy().sum(-1)

    def mode(self):
        return self.mean

class DiagGaussian(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(DiagGaussian, self).__init__()

        init_ = lambda m: init(m, nn.init.orthogonal_, lambda x: nn.init.
                               constant_(x, 0))

        self.fc_mean = init_(nn.Linear(num_inputs, num_outputs))
#         desired_init_log_std = -0.693471 #exp(..) ~= 0.5
#         desired_init_log_std = -1.609437 #exp(..) ~=0.2
        desired_init_log_std = -2.302585 #exp(..) ~=0.1
        
        self.logstd = AddBias(desired_init_log_std * torch.ones(num_outputs)) #so no state-dependent sigma

    def forward(self, x, mask=None):
        action_mean = self.fc_mean(x)
#         print('action_mean',action_mean.shape,x.shape)
        zeros = torch.zeros(action_mean.size())
        if x.is_cuda:
            zeros = zeros.to('cuda:3')

        action_logstd = self.logstd(zeros)
        return FixedNormal(action_mean, action_logstd.exp())

class ActionHead(nn.Module):
    def __init__(self, input_dim, output_dim, type="categorical"):
        super(ActionHead, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.type = type
        if type == "categorical":
            self.distribution = Categorical(num_inputs=input_dim, num_outputs=output_dim)
        elif type == "normal":
            self.distribution = DiagGaussian(num_inputs=input_dim, num_outputs=output_dim)
        else:
            raise NotImplementedError

    def forward(self, input, mask):
        if self.type == "normal":
            return self.distribution(input)
        else:
            return self.distribution(input, mask)
        
class Pi_net_mh(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Pi_net_mh, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, int(hidden_size/2))
        self.action_heads = nn.ModuleList()
        self.action_heads.append(ActionHead(int(hidden_size/2), 2, type='categorical'))
        self.action_heads.append(ActionHead(int(hidden_size/2)+1, 1, type='normal'))
        
    def forward(self, s, deterministic=False):
        x = F.relu(self.linear1(s))
        x = F.relu(self.linear2(x))
        
        action_outputs=[]
        head_outputs=[]
        head_outputs.append(x)
        action_type_dist = self.action_heads[0](x,mask=None)
        if deterministic:
            action_type = action_type_dist.mode()
        else:
            action_type = action_type_dist.sample()
            
        head_outputs.append(action_type)
        action_outputs.append(action_type)
        head_output = torch.cat(head_outputs, dim=-1)

        head_dist = self.action_heads[1](head_output,mask=None)
        
        if deterministic:
            head_action = head_dist.mode()
        else:
            head_action = head_dist.rsample()
        
        action_outputs.append(head_action)

        joint_action_log_prob = action_type_dist.log_probs(action_type)
        entropy = action_type_dist.entropy().mean()
        
        joint_action_log_prob += head_dist.log_probs(head_action)

        entropy += head_dist.entropy().mean()
        action_outputs = torch.cat(action_outputs,dim=-1)
        return action_outputs, joint_action_log_prob, entropy
    
actor_mh = Pi_net_mh(256, 256)
checkpoint = torch.load('./DRL-TU/egs/exp/DRL-TU-MH_pretrained/model_final.pkl',map_location='cpu')
actor_mh.load_state_dict(checkpoint['pi'])
# actor_mh = actor_mh.to('cuda:3')

In [ ]:
eers_v22={}
mindcts_v22={}

for tmp_scenario in tqdm(['random','1das','3das','5das','10das','1d1s','3d1s','5d1s','10d1s']):
# for tmp_scenario in tqdm(['random']):
    # 0. init
    scenario2model2day2score_v22[tmp_scenario]=defaultdict(list)
    
    # 1. Load trial trajectory and enrollment model
    f_read = open('./trials/%s/trial.pkl' %tmp_scenario, 'rb')
    trials = pickle.load(f_read)
    f_read.close()
    enrol_models={i.split()[0]:i.split()[1:] for i in open('./trials/%s/enrol_model'%tmp_scenario)}
    
    # 2. achieve enrol template
    enrol_embd_dict={}
    for tmp_model in enrol_models:
        enrol_embd=np.zeros((1,128))
        for utt in enrol_models[tmp_model]:
            enrol_embd += embds_dict[utt]

        enrol_embd_dict[tmp_model] = enrol_embd/5
    # 3. scoring
    true_score=[]
    false_score=[]
    
    if tmp_scenario in ['random']:
        for tmp_model in trials:
            enrol_embd = enrol_embd_dict[tmp_model]
            ts=tmp_model[3:7]
            scenario2model2day2score_v22[tmp_scenario][tmp_model]=defaultdict(list)
            for utt in trials[tmp_model]:
                test_embd = embds_dict[utt]
                observation = np.concatenate((enrol_embd,test_embd),axis=1)
                observation = torch.from_numpy(observation.astype('float32'))
#                 observation = observation.to('cuda:3')
                action_output, _, _ = actor_mh(observation,deterministic=True)
                action_output = action_output.cpu().detach().numpy()
                action_deter, action_value = action_output[0][0], action_output[0][1]

                result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                if ts == utt[:4]:
                    true_score.append(result) 
                    scenario2model2day2score_v22[tmp_scenario][tmp_model]['true'].append(result)
                else:
                    false_score.append(result) 
                    scenario2model2day2score_v22[tmp_scenario][tmp_model]['false'].append(result)
                    
                if action_deter==1 and result>0.4:
                    enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd
                    
    elif tmp_scenario in ['1das','3das','5das','10das']:
        for tmp_model in trials:
            enrol_embd = enrol_embd_dict[tmp_model]
            ts=tmp_model[3:7]
            scenario2model2day2score_v22[tmp_scenario][tmp_model]=defaultdict(list)
            for day in trials[tmp_model]:
                scenario2model2day2score_v22[tmp_scenario][tmp_model][day]=defaultdict(list)
                for utt in trials[tmp_model][day]:
                    test_embd = embds_dict[utt]
                    observation = np.concatenate((enrol_embd,test_embd),axis=1)
                    observation = torch.from_numpy(observation.astype('float32'))
#                     observation = observation.to('cuda:3')
                    action_output, _, _ = actor_mh(observation,deterministic=True)
                    action_output = action_output.cpu().detach().numpy()
                    action_deter, action_value = action_output[0][0], action_output[0][1]

                    result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                    if ts == utt[:4]:
                        true_score.append(result) 
                        scenario2model2day2score_v22[tmp_scenario][tmp_model][day]['true'].append(result)
                    else:
                        false_score.append(result) 
                        scenario2model2day2score_v22[tmp_scenario][tmp_model][day]['false'].append(result)
                        
                    if action_deter==1 and result>0.4:
                        enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd
                        
    elif tmp_scenario in ['1d1s','3d1s','5d1s','10d1s']:
        for tmp_round in trials:
            scenario2model2day2score_v22[tmp_scenario][tmp_round]=defaultdict(list)
            for tmp_model in trials[tmp_round]:
                enrol_embd = enrol_embd_dict[tmp_model]
                ts=tmp_model[3:7]
                scenario2model2day2score_v22[tmp_scenario][tmp_round][tmp_model]=defaultdict(list)
                for day in trials[tmp_round][tmp_model]:
                    scenario2model2day2score_v22[tmp_scenario][tmp_round][tmp_model][day]=defaultdict(list)
                    for utt in trials[tmp_round][tmp_model][day]:
                        test_embd = embds_dict[utt]
                        observation = np.concatenate((enrol_embd,test_embd),axis=1)
                        observation = torch.from_numpy(observation.astype('float32'))
#                         observation = observation.to('cuda:3')
                        action_output, _, _ = actor_mh(observation,deterministic=True)
                        action_output = action_output.cpu().detach().numpy()
                        action_deter, action_value = action_output[0][0], action_output[0][1]

                        result = 1 - spatial.distance.cosine(enrol_embd, test_embd)
                        if ts == utt[:4]:
                            true_score.append(result) 
                            scenario2model2day2score_v22[tmp_scenario][tmp_round][tmp_model][day]['true'].append(result)
                        else:
                            false_score.append(result) 
                            scenario2model2day2score_v22[tmp_scenario][tmp_round][tmp_model][day]['false'].append(result)
                            
                        if action_deter==1 and result>0.4:
                            enrol_embd = (1-action_value)*enrol_embd + action_value*test_embd

    eer_v22, threshold_eer, mindct_v22, threashold_dct = compute_eer(np.array(true_score), np.array(false_score))
    eers_v22[tmp_scenario]=eer_v22
    mindcts_v22[tmp_scenario]=mindct_v22
    print('---- Scenario : %s----'%tmp_scenario)
    print('EER: %s, EER_threshold: %s \nMinDCT: %s, MinDCT_threshold: %s, '%(eer_v22*100,threshold_eer, mindct_v22, threashold_dct))
    print(len(true_score),len(false_score))
    

 11%|██████████████████████████▏                                                                                                                                                                                                                 | 1/9 [04:14<33:55, 254.46s/it]

---- Scenario : random----
EER: 1.8745362475429257, EER_threshold: 0.4936059872128533 
MinDCT: 0.3013577370181856, MinDCT_threshold: 0.6593987079739003, 
76819 76819


In [ ]:
print('baseline:',eers_v0['random']*100)
for alpha in eers_v1:
    print(alpha,eers_v1[alpha]['random']*100)
print('Adw',eers_v21['random']*100)  
print('MH',eers_v22['random']*100)  


In [ ]:
for scenario in ['1das','3das','5das','10das','1d1s','3d1s','5d1s','10d1s']:
    print('-----%s-----'%scenario)
    print('baseline:',eers_v0[scenario]*100)
    for alpha in eers_v1:
        print(alpha,eers_v1[alpha][scenario]*100)
    print('Adw',eers_v21[scenario]*100)  
    print('MH',eers_v22[scenario]*100)  


In [ ]:
eer_dicts={'eers_v0':eers_v0,'eers_v1':eers_v1,'eers_v21':eers_v21,'eers_v22':eers_v22}

In [ ]:
np.save('eers.npy',eer_dicts)